In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *


import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from auctions.webutils import *


import gym
from gym import spaces
import numpy as np
from typing import Optional
from open_spiel.python.rl_environment import Environment

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from gym.envs.registration import register

register(
    id='gym_examples/OpenSpiel-v0',
    entry_point='gym_examples.envs:OpenSpielEnv',
    max_episode_steps=300,
)

In [34]:
import gym_examples
spiel_env_args = dict(chance_event_sampler=UBCChanceEventSampler(), all_simultaneous=True, terminal_rewards=True)
env = gym.make('gym_examples/OpenSpiel-v0', spiel_game_args={'filename': 'large_game_2.json'}, spiel_env_args=spiel_env_args)


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


In [35]:
### Single Env

observation, info = env.reset(return_info=True)
done = False
while not done:
    print("ACT")
    legal_actions = info['legal_actions']
    action = np.random.choice(legal_actions)
    print(legal_actions, action, info['current_player'])
    observation, reward, done, info = env.step(action)
print(reward, info['state_string'])

ACT
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63] 30 0
ACT
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63] 5 1
ACT
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 48, 49, 50, 52] 1 0
ACT
[1, 2, 3, 4, 5, 16] 1 1
-1 LinearValues:100, 70, 50 Budget: 500
LinearValues:100, 80, 60 Budget: 500
Price: 75, 55, 25
Round: 3
Player 0 processed: 1, 2, 2
Player 1 processed: 0, 1, 1
Aggregate demands: 1, 3, 3
Final bids: 1, 2, 2 | 0, 1, 1


In [42]:
### SyncVectorEnv

from gym.vector import SyncVectorEnv
from open_spiel.python.examples.ubc_utils import *

from gym.envs.registration import register
register(
    id='gym_examples/OpenSpiel-v0',
    entry_point='gym_examples.envs:OpenSpielEnv',
    max_episode_steps=300,
)

def make_spiel_game():
    env = gym.make('gym_examples/OpenSpiel-v0', spiel_game_args={'filename': 'large_game_2.json'}, spiel_env_args=spiel_env_args)
    return env
spiel_env_args = dict(chance_event_sampler=UBCChanceEventSampler(), all_simultaneous=True, terminal_rewards=True)

envs = gym.vector.SyncVectorEnv([make_spiel_game] * 2)
observations, infos = envs.reset(return_info=True)


i = 0
while True:
    actions = []
    for info in infos:
        legal_actions = info['legal_actions']
        action = np.random.choice(legal_actions)
        actions.append(action)
    observations, _, dones, infos = envs.step(actions)
    rewards = [info['rewards'] for info in infos]
    
    for q in range(len(infos)):
        print(infos[q]['state_string'], rewards[q])

    i += 1
    if i > 1000:
        break

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


Partial joint action: 21 
LinearValues:120, 80, 30 Budget: 500
LinearValues:150, 80, 30 Budget: 500
Price: 75, 50, 25
Round: 1
 [0, 0]
Partial joint action: 58 
LinearValues:120, 80, 30 Budget: 500
LinearValues:100, 80, 60 Budget: 500
Price: 75, 50, 25
Round: 1
 [0, 0]
Partial joint action: 
LinearValues:120, 80, 30 Budget: 500
LinearValues:150, 80, 30 Budget: 500
Price: 82.5, 50, 25
Round: 2
Player 0 processed: 1, 1, 1
Player 1 processed: 3, 1, 0
Aggregate demands: 4, 2, 1
 [0, 0]
Partial joint action: 
LinearValues:120, 80, 30 Budget: 500
LinearValues:100, 80, 60 Budget: 500
Price: 75, 55, 25
Round: 2
Player 0 processed: 3, 2, 2
Player 1 processed: 0, 2, 1
Aggregate demands: 3, 4, 3
 [0, 0]
Partial joint action: 5 
LinearValues:120, 80, 30 Budget: 500
LinearValues:150, 80, 30 Budget: 500
Price: 82.5, 50, 25
Round: 2
Player 0 processed: 1, 1, 1
Player 1 processed: 3, 1, 0
Aggregate demands: 4, 2, 1
 [0, 0]
Partial joint action: 2 
LinearValues:120, 80, 30 Budget: 500
LinearValues:100,

In [45]:
envs.action_space

MultiDiscrete([64 64])